In [ ]:
import pandas as pd
import numpy as np

csv_df = pd.read_csv('H28.csv',encoding="utf8") 
len(csv_df)
csv_df.info()
csv_df.head()

csv_df2 = pd.read_csv('H29.csv',encoding="utf8") 
len(csv_df2)
csv_df2.info()
csv_df2.head()

csv_df3 = pd.read_csv('H30.csv',encoding="utf8") 
len(csv_df3)
csv_df3.info()
csv_df3.head()

csv_df4 = pd.read_csv('R1.csv',encoding="utf8") 
len(csv_df4)
csv_df4.info()
csv_df4.head()

csv_df5 = pd.read_csv('R2.csv',encoding="utf8") 
len(csv_df5)
csv_df5.info()
csv_df5.head()

In [ ]:
def scaler(array, min_scale, max_scale):
    
    M = max(array)
    m = min(array)
    if M != m:
        scaled_array = (array - m*np.ones(shape = array.shape))/(M - m)
    else:
        scaled_array = array - m*np.ones(shape = array.shape)
    
    return scaled_array

def preprocessing(x):
    xl = []
    for i in range(len(x.T)):
        l = scaler(x.T[i], 0, 1)
        xl.append(l)
    
    return np.array(xl).T

def nearvalue(x, l):
    d = []
    for i in range(len(l)):
        d.append(abs(x-l[i]))
        
    return l[d.index(min(d))]

def r_2(v, w):
    
    s1 = np.sum((v-w)**2)
    s2 = np.sum((v-np.mean(v))**2)
    
    return 1-(s1/s2)

def accuracy(v,w):
    u = v-w
    a = 0
    for d in u:
        if d != 0:
            a+=1
            
    return 1-(a/len(u))

In [ ]:
import numpy as np
import time
import sklearn
ev = csv_df.drop(columns =['所在大字', '所在本番', '所在枝番', '所在小枝番', '所在地番区分',
                           '所在地番分割', '所在小字', '現況地目コード'])
ev2 = csv_df2.drop(columns =['所在大字', '所在本番', '所在枝番', '所在小枝番', '所在地番区分',
                           '所在地番分割', '所在小字', '現況地目コード'])
ev3 = csv_df3.drop(columns =['所在大字', '所在本番', '所在枝番', '所在小枝番', '所在地番区分',
                           '所在地番分割', '所在小字', '現況地目コード'])
ev4 = csv_df4.drop(columns =['所在大字', '所在本番', '所在枝番', '所在小枝番', '所在地番区分',
                           '所在地番分割', '所在小字', '現況地目コード'])
ev5 = csv_df5.drop(columns =['所在大字', '所在本番', '所在枝番', '所在小枝番', '所在地番区分',
                           '所在地番分割', '所在小字', '現況地目コード'])

x_H28 = np.array(ev.values)
y_H28 = np.array(csv_df['現況地目コード'].values)
x_H29 = np.array(ev2.values)
y_H29 = np.array(csv_df2['現況地目コード'].values)
x_H30 = np.array(ev3.values)
y_H30 = np.array(csv_df3['現況地目コード'].values)
x_R1 = np.array(ev4.values)
y_R1 = np.array(csv_df4['現況地目コード'].values)
x_R2 = np.array(ev5.values)
y_R2 = np.array(csv_df5['現況地目コード'].values)

x =np.concatenate([x_H28, x_H29])
x =np.concatenate([x, x_H30])
x =np.concatenate([x, x_R1])
x =np.concatenate([x, x_R2])
y =np.concatenate([y_H28, y_H29])
y =np.concatenate([y, y_H30])
y =np.concatenate([y, y_R1])
y =np.concatenate([y, y_R2])
y_class = list(set(y))

x = preprocessing(x)
x_H28 = x[0:len(x_H28)]
x_H29 = x[len(x_H28):len(x_H28)+len(x_H29)]
x_H30 = x[len(x_H28)+len(x_H29):len(x_H28)+len(x_H29)+len(x_H30)]
x_R1 = x[len(x_H28)+len(x_H29)+len(x_H30):len(x_H28)+len(x_H29)+len(x_H30)+len(x_R1)]
x_R2 = x[len(x_H28)+len(x_H29)+len(x_H30)+len(x_R1):]

In [ ]:
#特徴選択
from sklearn.feature_selection import SelectKBest, mutual_info_classif
def feature_selection(x_train,y_train,x_test,k):
    selector = SelectKBest(k=k, score_func=mutual_info_classif)
    selector.fit(x_train,y_train)
    x_train_new = selector.transform(x_train)
    x_test_new = selector.transform(x_test)
    print(x_train.shape)
    print(x_test.shape)
    print(x_train_new.shape)
    print(x_test_new.shape)
    
    return x_train_new, x_test_new

In [ ]:
#H28を学習しH29をテスト
x_train = x_H28
y_train = y_H28
x_test = x_H29
y_test = y_H29

x_train, x_test = feature_selection(x_train,y_train,x_test,10)

from sklearn import neural_network
clf = neural_network.MLPRegressor(activation="logistic", alpha=0.0001, max_iter=1000)

start = time.time()

clf.fit(x_train, y_train)

y_tr_predict = np.array([nearvalue(i, y_class) for i in clf.predict(x_train)])
y_predict = np.array([nearvalue(i, y_class) for i in clf.predict(x_test)])

from sklearn.metrics import mean_squared_error
import sklearn
r2_1 = r_2(y_test, y_predict)
accuracy1 = accuracy(y_predict,y_test)
print(f"決定係数{r2_1}")
print(f"正解率{accuracy1}")

accuracy1 = clf.score(x_test, y_test)
print(f"決定係数{accuracy1}")



#print("classification report")
#print(sklearn.metrics.classification_report(y_test, predicted))

t2 =time.time() - start
print("かかった時間")
print(t2)

In [ ]:
#H28とH29を学習しH30をテスト
x_train2 = np.concatenate([x_H28, x_H29])
y_train2 = np.concatenate([y_H28, y_H29])
x_test2 = x_H30
y_test2 = y_H30

x_train, x_test = feature_selection(x_train,y_train,x_test,10)

from sklearn import neural_network
clf2 = neural_network.MLPRegressor(activation="logistic", alpha=0.0001, max_iter=1000)

start = time.time()

clf2.fit(x_train2, y_train2)

y_tr_predict2 = np.array([nearvalue(i, y_class) for i in clf2.predict(x_train2)])
y_predict2 = np.array([nearvalue(i, y_class) for i in clf2.predict(x_test2)])

from sklearn.metrics import mean_squared_error
import sklearn
r2_2 = r_2(y_test2, y_predict2)
accuracy2 = accuracy(y_test2,y_predict2)
print(f"決定係数{r2_2}")
print(f"正解率{accuracy2}")

accuracy2 = clf2.score(x_test2, y_test2)
print(f"決定係数{accuracy2}")

#predicted2 = clf2.predict(x_test2)

#print("classification report")
#print(sklearn.metrics.classification_report(y_test2, predicted2))

t2 =time.time() - start
print("かかった時間")
print(t2)